# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Path hack.
import sys
import os

sys.path.append(os.path.abspath(".."))

In [3]:
import csv
import ast
from datetime import date, datetime, time, timedelta
from datetime import datetime
import pickle as pkl
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

import numpy as np
import pandas as pd
import sklearn
from joblib import dump, load

import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

from models.simple_nn import *
from utils.simple_nn_utils import *
from utils.general_utils import *
from utils.data_loader import *

# Read in Data

In [4]:
data = pd.read_pickle(
    f"/gpfs/milgram/project/rtaylor/shared/ABDPain_EarlyDiags/unq_pt_enc_clean_multilabel_nomismatches.pkl"
)

# Read Proc Codes In

## Investigate data file

In [83]:
proc_outcomes = pd.read_csv("/home/vs428/project/Data/procedure_outcome.csv")

In [84]:
proc_outcomes.groupby("PAT_ENC_CSN_ID").size().sort_values(ascending=False)

PAT_ENC_CSN_ID
234381633    7
146490372    7
253648708    6
100595317    6
116748038    6
            ..
154280223    1
154279999    1
154279520    1
154279141    1
274287190    1
Length: 364830, dtype: int64

In [85]:
PAT_ENC_CSN_ID = 234381633
display(data[data["PAT_ENC_CSN_ID"] == PAT_ENC_CSN_ID])
display(proc_outcomes[proc_outcomes["PAT_ENC_CSN_ID"] == PAT_ENC_CSN_ID])

,PAT_ENC_CSN_ID,person_id,visit_occurrence_id,age,EdDisposition_AMA,EdDisposition_Admit,EdDisposition_Discharge,EdDisposition_Eloped,EdDisposition_Expired,EdDisposition_LWBS after Triage,...,last_BP_Diastolic,ed_SpO2,ed_Temp,ed_Patient Acuity,ed_Pulse,ed_Pain Score,ed_Resp,ed_BP_Systolic,ed_BP_Diastolic,multilabel
148127,234381633,7366201,220910288,13.0,NaN,1.0,1.0,NaN,NaN,NaN,...,62.0,98.0,98.8,3.0,64.0,7.0,16.0,103.0,54.0,[Hepatitis]


,PAT_ENC_CSN_ID,Code,DISP_NAME
18933,234381633,IMG3479,US NON-OB PELVIS COMPLETE WITH LIMITED DOPPLER
145389,234381633,IMG524,US ABDOMEN COMPLETE
163709,234381633,IMG1219,US DUPLEX ABDOMEN PELVIS VASCULAR STUDY COMPLE...
170514,234381633,IMG794,CT ABDOMEN PELVIS W IV CONTRAST
242074,234381633,IMG2320,US ABDOMEN LIMITED
296910,234381633,IMG1220,US DUPLEX ABDOMEN PELVIS STUDY LIMITED (BH YH ...
433312,234381633,IMG550,US PELVIS LIMITED


## Create Label column

In [86]:
proc_outcomes["Test_Category"] = proc_outcomes["DISP_NAME"].str[:2]

In [87]:
proc_outcomes["Test_Category"].value_counts()

CT    307520
US    153217
Name: Test_Category, dtype: int64

In [88]:
proc_outcomes.columns

Index(['PAT_ENC_CSN_ID', 'Code', 'DISP_NAME', 'Test_Category'], dtype='object')

In [89]:
labeled_proc_outcomes = proc_outcomes.groupby("PAT_ENC_CSN_ID").apply(
    lambda x: "both"
    if (("CT" in x["Test_Category"].values) and ("US" in x["Test_Category"].values))
    else x["Test_Category"].unique()[0]
)

In [90]:
# 460737
proc_outcomes = proc_outcomes.merge(labeled_proc_outcomes.reset_index())
proc_outcomes = proc_outcomes.rename({0: "proc_label"}, axis=1)

In [98]:
# make sure that there is only one label per PAT_ENC_CSN_ID, despite multiple rows
assert (
    proc_outcomes.groupby("PAT_ENC_CSN_ID").agg({"proc_label": "nunique"}) > 1
).sum() == 0

proc_label    0
dtype: int64

In [99]:
merged = data.merge(
    proc_outcomes.groupby("PAT_ENC_CSN_ID").first(),
    on="PAT_ENC_CSN_ID",
    how="left",
)

In [105]:
merged["proc_label"] = merged["proc_label"].fillna("none")

In [111]:
merged.to_pickle(
    f"/gpfs/milgram/project/rtaylor/shared/ABDPain_EarlyDiags/unq_pt_enc_clean_multilabel_nomismatches_CTUS.pkl"
)